In [2]:
import qiskit 
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
# from qiskit.visualization import *
# from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
# from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
# service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [3]:
from qiskit import *
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import math

In [4]:
def adder(x, y):
    n = math.ceil(math.log2(x))
    m = math.ceil(math.log2(y))
    
    # t = math.min(n, m)
    # m = math.max(n, m)
    # n = t;
    
    a = QuantumRegister(m, 'a') #First number
    b = QuantumRegister(m + 1, 'b') #Second number and sum
    c = QuantumRegister(m, 'c') #Carry bits
        
    cl = ClassicalRegister(m + 1, 'cl') #Final output
    
    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(a, b, c, cl)
    
    # Modifying qubits according to input values
    i = 0
    while(x != 0):
        if(x % 2 == 1):
            qc.x(a[i])
            print(1)
        x = math.floor(x/2)
        # print(x)
        i = i + 1
    i = 0
    while(y != 0):
        if(y % 2 == 1):
            qc.x(b[i])
        y = math.floor(y/2)
        i = i + 1
    
    # print(qc)
    
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 1):
        # print(i)
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[ i], c[i+1])
    
    # print(qc)
    
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(a[m - 1], b[m - 1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(c[m-1], b[m-1], b[m])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #
    qc.cx(a[m - 1], b[m - 1])
    qc.cx(c[m - 1], b[m - 1])
    
    for i in range(m - 2, -1, -1):
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(c[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i + 1])
        
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(a[i], b[i])
        qc.cx(c[i], b[i])
    
    #Measure qubits
    for i in range(m + 1):
        qc.measure(b[i], cl[i])
    print(qc)
        
    #Set chosen backend and execute job
    backend = AerSimulator()
    
    # First we have to transpile the quantum circuit 
    # to the low-level QASM instructions used by the 
    # backend
    qc_compiled = transpile(qc, backend)
    
    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend.run(qc_compiled, shots=1024)
    
    # Grab the results from the job.
    result_sim = job_sim.result()
    
    counts = result_sim.get_counts(qc_compiled)
    print(counts)
    
#     plot_histogram(counts)
    
adder(4, 6)

1
                                                                            »
 a_0: ───────■────■─────────────────────────────────────────────────────────»
             │    │                                                         »
 a_1: ───────┼────┼────■──────────────■─────────────────────────────────────»
      ┌───┐  │    │    │              │                                     »
 a_2: ┤ X ├──┼────┼────┼────■─────────┼────■──────────────■────■────────────»
      └───┘  │  ┌─┴─┐  │    │         │    │              │    │            »
 b_0: ───────■──┤ X ├──┼────┼────■────┼────┼──────────────┼────┼────────────»
      ┌───┐  │  └───┘  │    │    │  ┌─┴─┐  │              │    │            »
 b_1: ┤ X ├──┼─────────■────┼────┼──┤ X ├──┼────■─────────┼────┼─────────■──»
      ├───┤  │         │    │    │  └───┘┌─┴─┐  │       ┌─┴─┐┌─┴─┐┌───┐  │  »
 b_2: ┤ X ├──┼─────────┼────■────┼───────┤ X ├──┼────■──┤ X ├┤ X ├┤ X ├──┼──»
      └───┘  │         │  ┌─┴─┐  │       └───┘  │  ┌─┴─┐└┬─┬┘└

In [7]:
def adder(x, y):
    n = math.ceil(math.log2(x))
    m = math.ceil(math.log2(y))
    
    # t = math.min(n, m)
    # m = math.max(n, m)
    # n = t;
    
    print(n)
    print(m)
    
    a = QuantumRegister(m, 'a') #First number
    b = QuantumRegister(m + 1, 'b') #Second number and sum
    c = QuantumRegister(m, 'c') #Carry bits
        
    cl = ClassicalRegister(m + 1, 'cl') #Final output
    
    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(a, b, c, cl)
    
    # Modifying qubits according to input values
    i = 0
    while(x != 0):
        if(x % 2 == 1):
            qc.x(a[i])
            print(1)
        x = math.floor(x/2)
        # print(x)
        i = i + 1
    i = 0
    while(y != 0):
        if(y % 2 == 1):
            qc.x(b[i])
        y = math.floor(y/2)
        i = i + 1
    
    
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 1):
        # print(i)
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i+1])
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(a[m - 1], b[m - 1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(c[m-1], b[m-1], b[m])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #
    qc.cx(a[m - 1], b[m - 1])
    qc.cx(c[m - 1], b[m - 1])
    
    for i in range(m - 2, -1, -1):
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(c[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i + 1])
        
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(a[i], b[i])
        qc.cx(c[i], b[i])
    
    #Measure qubits
    for i in range(m + 1):
        qc.measure(b[i], cl[i])
    print(qc)
        
    #Set chosen backend and execute job
    backend = AerSimulator()
    
    # First we have to transpile the quantum circuit 
    # to the low-level QASM instructions used by the 
    # backend
    qc_compiled = transpile(qc, backend)
    
    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend.run(qc_compiled, shots=1024)
    
    # Grab the results from the job.
    result_sim = job_sim.result()
    
    counts = result_sim.get_counts(qc_compiled)
    print(counts)

    
#     plot_histogram(counts)
    
adder(4, 6)

2
3
1
                                                                            »
 a_0: ───────■────■─────────────────────────────────────────────────────────»
             │    │                                                         »
 a_1: ───────┼────┼────■──────────────■─────────────────────────────────────»
      ┌───┐  │    │    │              │                                     »
 a_2: ┤ X ├──┼────┼────┼────■─────────┼────■──────────────■────■────────────»
      └───┘  │  ┌─┴─┐  │    │         │    │              │    │            »
 b_0: ───────■──┤ X ├──┼────┼────■────┼────┼──────────────┼────┼────────────»
      ┌───┐  │  └───┘  │    │    │  ┌─┴─┐  │              │    │            »
 b_1: ┤ X ├──┼─────────■────┼────┼──┤ X ├──┼────■─────────┼────┼─────────■──»
      ├───┤  │         │    │    │  └───┘┌─┴─┐  │       ┌─┴─┐┌─┴─┐┌───┐  │  »
 b_2: ┤ X ├──┼─────────┼────■────┼───────┤ X ├──┼────■──┤ X ├┤ X ├┤ X ├──┼──»
      └───┘  │         │  ┌─┴─┐  │       └───┘  │  ┌─┴─┐└┬